Dany jest graf skierowany G = (V,E), funkcja c: E -> N dająca wagi krawędziom, oraz wyróżnione wierzchołki s i t. Należy znaleźć maksymalny przepływ w grafie G pomiędzy s i t, tzn. funkcję f: E -> N spełniającą warunki definicji przepływu, zapewniającą największą przepustowość.

Do rozwiązania zadania należy wykorzystać algorytm Forda-Fulkersona, porównując dwie strategie znajdowania ścieżek powiększających:

- przy użyciu przeszukiwania metodą DFS
- przy użyciu przeszukiwania metodą BFS (algorytm Edmondsa-Karpa)

Importy niezbędnych bibliotek

In [11]:
from dimacs import *
from queue import deque
import time
import os

In [4]:
def readSolution(graph):
    with open(graph, "r") as f:
        first_line = f.readline()
    return int(first_line.split("=")[1])

def compareResults(file_path, graph_name, method, show_results=False):
    """Compares the result from the file with the result from the given method."""
    file_solution = readSolution(file_path)
    algorithm_solution = method(file_path)

    print("Graph name:", graph_name)
    if file_solution == algorithm_solution:
        print("Test passed")
    else:
        print("Test failed")
        show_results = True

    if show_results:
        print("Solution from file: ", file_solution)
        print("Solution from algorithm: ", algorithm_solution)

def testAllGraphs(directory_path, method):
    """Tests all graphs in the given directory with the specified method."""
    for filename in os.listdir(directory_path):
        if filename == 'grid100x100':
            continue

        file_path = os.path.join(directory_path, filename)
        if os.path.isfile(file_path):
            compareResults(file_path, filename, method)

1. Konwersja grafu na strukturę pozwalającą na wygodną implementację algorytmu Forda-Fulkersona. Lista sąsiedztwa zapewnia dostęp do sąsiadów, macierz pozwala na odwoływanie się do pojemności krawędzi tak w grafie oryginalnym, jak i residualnym.

In [5]:
def convert_graph(graph):
    n, graph = loadWeightedGraph(graph)

    G = [[] for _ in range(n)]
    matrix = [[0 for _ in range(n)] for _ in range(n)]


    for (u, v, w) in graph:
        u -= 1
        v -= 1

        G[u].append(v)
        G[v].append(u)
        matrix[u][v] = w

    return G, matrix

2.1. Znajdowanie ścieżki powiększającej metodą DFS

In [6]:
def find_path_dfs(graph, matrix, s, t, path = []):
    if s == t:
        if len(path) == 1:
            path = [0] + path
        return path

    for v in graph[s]:
        if matrix[s][v] > 0 and v not in path:
            new_path = find_path_dfs(graph, matrix, v, t, path + [v])
            if new_path is not None:
                return new_path

    return None

2.2. Znajdowanie ścieżki powiększającej metodą BFS

In [7]:
def find_path_bfs(graph, matrix, s, t):
    q = deque()
    q.append(s)

    parent = [None for _ in range(len(graph))]
    parent[s] = s

    while len(q) > 0:
        u = q.popleft()

        for v in graph[u]:
            if matrix[u][v] > 0 and parent[v] == None:
                parent[v] = u
                q.append(v)

    if parent[t] == None:
        return None

    path = []
    u = t
    while u != s:
        path.append(u)
        u = parent[u]

    path.append(s)
    path.reverse()

    return path

Algorytm Forda-Fulkersona

In [8]:
def ford_fulkerson(graph, path_finder):
    graph, matrix = convert_graph(graph)
    n = len(graph)
    s, t = 0, n -1
    flow = 0

    path = path_finder(graph, matrix, s, t)
    while path:
        if len(path) == 1:
            path = [0] + path

        min_capacity = min(matrix[u][v] for u, v in zip(path, path[1:]))
        
        flow += min_capacity

        for u, v in zip(path, path[1:]):
            matrix[u][v] -= min_capacity
            matrix[v][u] += min_capacity

        path = path_finder(graph, matrix, s, t)
    return flow

In [13]:
ford_fulkerson("graphs/flow/grid100x100", find_path_bfs) 

4179